# Modifies data from TOGA* as specified below

In [ ]:
# The dataset is located at : RQ2/toga-model-inputs-outputs. 
# input.csv that's the input file where each row contains three information 
    # 1. focus method (the method being tested) 
    # 2, a test case with test assertions (you will need to remove that) 
    # 3) docstring (not necessary for SEER).
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

folder_dir = './toga-model-inputs-outputs'

In [ ]:
colnames = ['method', 'test', 'docstring']
df = pd.DataFrame(columns=colnames)
project_dict = {}

folder_names = os.listdir(folder_dir)
for project in folder_names:
    project_path = os.path.join(folder_dir, project)
    temp_df = pd.read_csv(project_path + '/inputs.csv')
    temp_df.columns = colnames
    temp_df = temp_df.drop('docstring', axis=1)
    temp_df['test'] = temp_df.apply(lambda row: re.sub(r'\s*assert.*', "", row['test']), axis=1)
    temp_df.to_csv(project_path + '/inputs_modified.csv')
    project_dict[project] = temp_df